In [1]:
from caravan import Caravan
caravan = Caravan()
caravan.group("speech").email("prabhune@berkeley.edu").key("BYxUTe7n").gpu_count(1).build()

Verifying group access...
Setting up a new caravan of GPU(s)...
Received a caravan of GPU(s)!
Connecting to each GPU in your caravan...
Connected to each GPU in your caravan!
Built Caravan!


In [3]:
import torch
import numpy as np

In [4]:
torch.nn.Transformer

torch.nn.modules.transformer.Transformer

In [5]:
model = torch.nn.Transformer()

/Users/prabhune/thava/client/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [6]:
model = model.to(0)

In [7]:
x = torch.randn(10, 32, 512).to(0)  # (sequence_length, batch_size, embedding_dim)
print(x.shape)
y = torch.randn(10, 32, 512).to(0)  # (sequence_length, batch_size, embedding_dim)
print(y.shape)
output = model(x, y)
print(output.shape)  # (sequence_length, batch_size, embedding_dim)

torch.Size([10, 32, 512])
torch.Size([10, 32, 512])


KeyboardInterrupt: 

In [6]:
a = torch.tensor([1, 2, 3], device=0)

In [7]:
a

tensor([1, 2, 3], device='mps:0')

In [8]:
a.to(device=0)

tensor([1, 2, 3], device='mps:0')

In [9]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = torch.nn.Linear(8, 1)
        self.act1 = torch.nn.ReLU()
        self.hidden2 = torch.nn.Linear(12, 8)
        self.act2 = torch.nn.ReLU()
        self.output = torch.nn.Linear(8, 1)
        self.act_output = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.hidden1(x)
        # x = self.act1(self.hidden1(x))
        # x = self.act2(self.hidden2(x))
        # x = self.act_output(self.output(x))
        return x

In [10]:
mymodule = Model().to(device=0)

In [11]:
type(mymodule)

__main__.Model

In [12]:
a = torch.tensor([[1.0, 2, 3, 4, 5, 6, 7, 8]], device=0)

In [13]:
out = mymodule(a)

Serialize: 7.8e-05
IPC: 0.19710493087768555
Deserialize: 0.00017189979553222656



In [14]:
type(out)

rtorch.biject.RTensor

In [15]:
print(out.rid)
print(out)

RId(id=2240209448702670)
Serialize: 4.5e-05
IPC: 0.08270430564880371
Deserialize: 4.0531158447265625e-06

tensor([[-1.4626]], device='mps:0', grad_fn=<LinearBackward0>)


In [16]:
torch.nn.BCELoss.__call__

<function torch.nn.modules.module.Module._wrapped_call_impl(self, *args, **kwargs)>

In [17]:
mymodule.parameters()

<generator object Module.parameters at 0x135e14ac0>

In [18]:
torch.optim.Adam.__init__

<function torch.optim.adam.Adam.__init__(self, params: Union[Iterable[torch.Tensor], Iterable[Dict[str, Any]]], lr: Union[float, torch.Tensor] = 0.001, betas: Tuple[float, float] = (0.9, 0.999), eps: float = 1e-08, weight_decay: float = 0, amsgrad: bool = False, *, foreach: Optional[bool] = None, maximize: bool = False, capturable: bool = False, differentiable: bool = False, fused: Optional[bool] = None)>

In [19]:
optimizer = torch.optim.Adam(mymodule.parameters(), lr=0.01)

In [20]:
type(optimizer)

torch.optim.adam.Adam

In [21]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [22]:
optimizer.step()

Serialize: 5.6e-05
IPC: 0.6681473255157471
Deserialize: 4.458427429199219e-05



In [23]:
optimizer.zero_grad()

In [24]:
a = torch.tensor([0.1, 0.2, 0.3], device=0)
b = torch.tensor([0.1, 0.2, 0.3], device=0)

In [25]:
loss_fn = torch.nn.MSELoss()

In [26]:
loss_fn(a, b)

Serialize: 6.2e-05
IPC: 0.12306618690490723
Deserialize: 4.887580871582031e-05



tensor(0., device='mps:0')

In [27]:
a = torch.tensor([0.1, 0.2, 0.3])
b = torch.tensor([0.1, 0.2, 0.3])

In [28]:
loss_fn(a, b)

tensor(0.)

In [29]:
a = torch.tensor([1,2,3,4,5], device=0)
b = torch.tensor([2,3,4,5,6], device=0)
c = a + b

Serialize: 1.8e-05
IPC: 0.06506967544555664
Deserialize: 6.222724914550781e-05



In [30]:
device = 0

dataset = np.loadtxt('pima-indians.csv', delimiter=',')
X = dataset[:,0:8]
print(X.shape)
y = dataset[:,8]

X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device).reshape(-1,1)

model = Model().to(device=device)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

(768, 8)
49413
Serialize: 0.00018
IPC: 1.054084062576294
Deserialize: 6.008148193359375e-05



In [31]:
n_epochs = 200
batch_size = 256

for epoch in range(n_epochs):
    for i in range(0, 768, batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Finished epoch {epoch}, latest loss {loss}, id {loss.rid}')

Serialize: 4.2e-05
IPC: 0.13765978813171387
Deserialize: 5.2928924560546875e-05

Serialize: 0.00011
IPC: 0.13586115837097168
Deserialize: 1.0013580322265625e-05

Finished epoch 0, latest loss tensor(5032.7725, device='mps:0', grad_fn=<MseLossBackward0>), id RId(id=2558771468014304)
Finished epoch 1, latest loss tensor(4262.5713, device='mps:0', grad_fn=<MseLossBackward0>), id RId(id=4302313441825504)
Finished epoch 2, latest loss tensor(3559.2476, device='mps:0', grad_fn=<MseLossBackward0>), id RId(id=4022118365369056)
Finished epoch 3, latest loss tensor(2922.8027, device='mps:0', grad_fn=<MseLossBackward0>), id RId(id=3689735141298912)
Finished epoch 4, latest loss tensor(2353.2368, device='mps:0', grad_fn=<MseLossBackward0>), id RId(id=375691131065056)
Finished epoch 5, latest loss tensor(1850.5488, device='mps:0', grad_fn=<MseLossBackward0>), id RId(id=2185251047183072)
Finished epoch 6, latest loss tensor(1414.7397, device='mps:0', grad_fn=<MseLossBackward0>), id RId(id=4439425977